In [1]:
!pip install newspaper3k lxml_html_clean
!pip install --upgrade lxml
!pip install sastrawi

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 5.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=e3a653c70af63f64c5a31f2c36c1026680908fc9861ba7ea6fdbfe254e3dcd63
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=3cd381000f1586e26b5d72f8e3ea11fef63972a42e942fcac55d0da4f088c584
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [2]:
!pip install newspaper3k

In [4]:
import feedparser
import pandas as pd
import nltk, re, html
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from wordcloud import WordCloud

nltk.download("punkt")
nltk.download("punkt_tab")

# ============================
# 1. AMBIL DATA DARI GOOGLE NEWS
# ============================

RSS = "https://news.google.com/rss/search?q=%22kesehatan+mental%22+OR+%22depresi%22+OR+%22gangguan+mental%22&hl=id&gl=ID&ceid=ID:id"

feed = feedparser.parse(RSS)

titles = []
summaries = []
links = []

for entry in feed.entries:
    titles.append(entry.title)
    summaries.append(entry.summary)
    links.append(entry.link)

print("Total artikel ditemukan:", len(titles))

df = pd.DataFrame({
    "title": titles,
    "summary": summaries,
    "link": links
})

df["raw_text"] = df["title"] + " " + df["summary"]

# ============================
# 2. CLEAN HTML & ENTITIES
# ============================

def clean_html(text):
    # hilangkan HTML tags
    text = BeautifulSoup(text, "html.parser").get_text(" ")
    # decode html entities (&nbsp;, &quot;, dll)
    text = html.unescape(text)
    # hapus hex warna/angka (misal 6f6f6f)
    text = re.sub(r"\b[0-9a-fA-F]{6}\b", " ", text)
    # hapus kata font, color, news, google bila muncul dalam HTML
    text = re.sub(r"\b(font|color|google|news|com|href|rss|articles|nbsp)\b", " ", text)
    # bersihkan karakter non-alfanumerik
    text = re.sub(r"[^A-Za-z0-9\u00C0-\u024F ]", " ", text)
    return text

df["clean_html"] = df["raw_text"].apply(clean_html)

# ============================
# 3. PREPROCESS NLP
# ============================

factory = StemmerFactory()
stemmer = factory.create_stemmer()

stopwords = set([
    "yang","dan","di","ke","dari","pada","untuk","dengan","sebagai","itu","ini","atau","oleh",
    "para","adalah","juga","tidak","saat","akan","karena","lebih","agar","bagi","dalam",
    "kita","mereka","akan","sudah","belum","jadi","hingga","sehat","mental","kesehatan"
])

def preprocess(text):
    t = text.lower()
    tokens = nltk.word_tokenize(t)
    tokens = [stemmer.stem(w) for w in tokens if w not in stopwords and len(w) > 2]
    return " ".join(tokens)

df["clean"] = df["clean_html"].apply(preprocess)
df.to_csv("cleaned_articles.csv", index=False)

# ============================
# 4. TF-IDF
# ============================

vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1,2))
X = vectorizer.fit_transform(df["clean"])

scores = np.asarray(X.mean(axis=0)).ravel()
terms = vectorizer.get_feature_names_out()

top_idx = scores.argsort()[::-1][:20]
top_df = pd.DataFrame({"term": terms[top_idx], "score": scores[top_idx]})

print("\nTOP 20 TF-IDF:")
print(top_df)

# ============================
# 5. WORDCLOUD
# ============================

wc = WordCloud(width=1200, height=600, background_color="white")
wc.generate(" ".join(df["clean"]))
wc.to_file("wordcloud.png")

print("\nWordcloud berhasil dibuat: wordcloud.png")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Total artikel ditemukan: 100

TOP 20 TF-IDF:
             term     score
0         depresi  0.036340
1            rata  0.022247
2       indonesia  0.019078
3            jaga  0.017196
4       mahasiswa  0.015978
5            kuat  0.015964
6          remaja  0.015402
7         jakarta  0.015044
8          antara  0.014744
9           warga  0.013822
10           news  0.013564
11         ganggu  0.013325
12           anak  0.012688
13  angka depresi  0.012542
14          angka  0.012542
15       nasional  0.012523
16    antara news  0.012285
17          alami  0.012132
18        tingkat  0.012027
19           guru  0.011837

Wordcloud berhasil dibuat: wordcloud.png


In [ ]:
from google.colab import files
files.download("wordcloud.png")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>